### **This notebook will attempt to remove outliers from each group** 

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.model_selection import cross_validate, GroupKFold
from sklearn.metrics import accuracy_score
from Spectra_Preparation_Functions import *

In [4]:
spectra_df = pd.read_csv("../../data/current_clean_spectrum.csv")
df = spectra_df

In [3]:
df['Absorbance'] = spectra_df.groupby('SpecID')['Absorbance'].transform(lambda x: normalise(x))

NameError: name 'df' is not defined

In [ ]:
#sns.lineplot(data=df, x='WaveNumber', y='Absorbance', hue='Status', errorbar=('ci', False))

In [ ]:
df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
0,201210-1-00,293,400.22778,201210-1,Normal,0.241429
1,201210-1-00,294,400.91116,201210-1,Normal,0.241086
2,201210-1-00,295,401.59454,201210-1,Normal,0.240729
3,201210-1-00,296,402.27789,201210-1,Normal,0.240359
4,201210-1-00,297,402.96127,201210-1,Normal,0.239978
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,0.060284
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,0.064627
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,0.069153
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,0.073864


Test on one surface

In [ ]:
#df = df[df['SurID'] == '201210-1']

##### **Function to apply quartiles calculation for each SurID group, then by WaveNumber**


In [ ]:
# Correctly define the function to apply multiple aggregations

def get_surface_wavenumber_quartiles(df):

    # Apply aggregation directly within groupby
    grouped = df.groupby(['SurID', 'WaveNumber']).agg(
        Lower_Quartile=('Absorbance', lambda x: x.quantile(0.25)),
        Median=('Absorbance', 'median'),
        Upper_Quartile=('Absorbance', lambda x: x.quantile(0.75))
    ).reset_index()
    return grouped

# Apply the function to your DataFrame
quartile_data = get_surface_wavenumber_quartiles(df)

# Calculate the IQR
quartile_data['IQR'] = quartile_data['Upper_Quartile'] - quartile_data['Lower_Quartile']

In [ ]:
quartile_data

,SurID,WaveNumber,Lower_Quartile,Median,Upper_Quartile,IQR
0,201210-1,400.22778,-0.031459,0.031138,0.115584,0.147043
1,201210-1,400.91116,-0.026116,0.029919,0.100635,0.126751
2,201210-1,401.59454,-0.030749,0.028575,0.095893,0.126642
3,201210-1,402.27789,-0.027876,0.023878,0.091889,0.119765
4,201210-1,402.96127,-0.029666,0.018957,0.087505,0.117171
...,...,...,...,...,...,...
129082,210526-3,1797.03870,-0.009718,0.018055,0.051074,0.060792
129083,210526-3,1797.72200,-0.008041,0.019022,0.057708,0.065748
129084,210526-3,1798.40550,-0.004021,0.020320,0.055508,0.059529
129085,210526-3,1799.08890,-0.000584,0.021803,0.059230,0.059814


Now we shall remove outliers spectra that have an absorbance outside 1.5 times the IQR at each WaveNumber in their group.

In [ ]:
merged_df = df.merge(quartile_data, on=['SurID', 'WaveNumber'])
merged_df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance,Lower_Quartile,Median,Upper_Quartile,IQR
0,201210-1-00,293,400.22778,201210-1,Normal,0.241429,-0.031459,0.031138,0.115584,0.147043
1,201210-1-00,294,400.91116,201210-1,Normal,0.241086,-0.026116,0.029919,0.100635,0.126751
2,201210-1-00,295,401.59454,201210-1,Normal,0.240729,-0.030749,0.028575,0.095893,0.126642
3,201210-1-00,296,402.27789,201210-1,Normal,0.240359,-0.027876,0.023878,0.091889,0.119765
4,201210-1-00,297,402.96127,201210-1,Normal,0.239978,-0.029666,0.018957,0.087505,0.117171
...,...,...,...,...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,0.060284,-0.009718,0.018055,0.051074,0.060792
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,0.064627,-0.008041,0.019022,0.057708,0.065748
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,0.069153,-0.004021,0.020320,0.055508,0.059529
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,0.073864,-0.000584,0.021803,0.059230,0.059814


In [ ]:
# Filter rows where Absorbance is an outlier
outliers = merged_df[
    (merged_df['Absorbance'] < (merged_df['Lower_Quartile'] - 1.5 * merged_df['IQR'])) |
    (merged_df['Absorbance'] > (merged_df['Upper_Quartile'] + 1.5 * merged_df['IQR']))
]

outliers

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance,Lower_Quartile,Median,Upper_Quartile,IQR
303,201210-1-00,596,607.28931,201210-1,Normal,0.725961,0.432922,0.483872,0.543991,0.111069
304,201210-1-00,597,607.97266,201210-1,Normal,0.737907,0.437763,0.482185,0.556428,0.118665
306,201210-1-00,599,609.33942,201210-1,Normal,0.762332,0.444864,0.490573,0.569483,0.124619
307,201210-1-00,600,610.02277,201210-1,Normal,0.768127,0.451626,0.488760,0.568165,0.116539
308,201210-1-00,601,610.70618,201210-1,Normal,0.772742,0.450023,0.489074,0.573122,0.123099
...,...,...,...,...,...,...,...,...,...,...
6237431,210526-3-09,568,588.15491,210526-3,Hyperglycemia,0.041477,0.160319,0.197802,0.238065,0.077746
6237432,210526-3-09,569,588.83826,210526-3,Hyperglycemia,0.041704,0.160581,0.201098,0.238386,0.077805
6237433,210526-3-09,570,589.52167,210526-3,Hyperglycemia,0.046057,0.162069,0.205352,0.237846,0.075777
6238845,210526-3-09,1982,1554.44190,210526-3,Hyperglycemia,0.574407,0.808342,0.904848,0.958891,0.150549


**If a defined proportion of a spectra is an outlier drop it from the dataframe**

In [ ]:
spectra_length = df['WaveNumber'].nunique()  # This should be consistent for all spectra
spectra_length

2049

In [ ]:
outlier_proportions = outliers['SpecID'].value_counts()/spectra_length
outlier_proportions

SpecID
210419-2-27    0.747194
210419-2-26    0.723280
210419-2-25    0.683260
210415-2-23    0.551489
210419-2-28    0.525134
                 ...   
201210-2-29    0.000488
210524-1-42    0.000488
210407-2-48    0.000488
201210-1-22    0.000488
210114-1-12    0.000488
Name: count, Length: 2491, dtype: float64

**Set the proportion of outliers needed to drop this spectra.**

In [ ]:
proportion_cutoff = 0.025
outlier_SpecIDs = outlier_proportions[outlier_proportions > proportion_cutoff].index
print(outlier_SpecIDs)
print(len(outlier_SpecIDs))

Index(['210419-2-27', '210419-2-26', '210419-2-25', '210415-2-23',
       '210419-2-28', '210415-1-00', '210504-2-31', '210419-2-30',
       '210428-2-07', '210504-2-41',
       ...
       '210324-2-15', '210415-2-25', '210303-1-24', '210324-1-15',
       '210303-1-19', '210401-1-29', '210304-1-24', '210324-1-45',
       '210311-1-35', '210428-2-08'],
      dtype='object', name='SpecID', length=1252)
1252


Drop the outler spectra from the dataset

In [ ]:
clean_df = spectra_df[~spectra_df['SpecID'].isin(outlier_SpecIDs)]

In [ ]:
clean_df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
14343,201210-1-07,293,400.22778,201210-1,Normal,39.462860
14344,201210-1-07,294,400.91116,201210-1,Normal,42.711925
14345,201210-1-07,295,401.59454,201210-1,Normal,45.566652
14346,201210-1-07,296,402.27789,201210-1,Normal,48.047821
14347,201210-1-07,297,402.96127,201210-1,Normal,50.176210
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,12.378163
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,13.269937
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,14.199285
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,15.166531


In [ ]:
# clean_df.to_csv("../../data/outlier_spectra_removed.csv", index=False)

#### **Check model performance when outliers are removed**

In [ ]:
clean_df

,SpecID,Seq,WaveNumber,SurID,Status,Absorbance
14343,201210-1-07,293,400.22778,201210-1,Normal,39.462860
14344,201210-1-07,294,400.91116,201210-1,Normal,42.711925
14345,201210-1-07,295,401.59454,201210-1,Normal,45.566652
14346,201210-1-07,296,402.27789,201210-1,Normal,48.047821
14347,201210-1-07,297,402.96127,201210-1,Normal,50.176210
...,...,...,...,...,...,...
6239200,210526-3-09,2337,1797.03870,210526-3,Hyperglycemia,12.378163
6239201,210526-3-09,2338,1797.72200,210526-3,Hyperglycemia,13.269937
6239202,210526-3-09,2339,1798.40550,210526-3,Hyperglycemia,14.199285
6239203,210526-3-09,2340,1799.08890,210526-3,Hyperglycemia,15.166531


In [ ]:
def prepare_wavelength_df(df, absorbance_col, status_col='Status'):

    # Pivot the DataFrame to get wavelengths as columns and absorbance values
    wavelength_df = df.pivot(index='SpecID', columns='WaveNumber', values=absorbance_col).reset_index()
    wavelength_df.columns.name = None

    # Merge with the statuses based on SpecID
    # Include the SurID to perform GroupKFold CV
    statuses_and_surface = df[['SpecID', 'SurID', status_col]].drop_duplicates()
    wavelength_df = pd.merge(wavelength_df, statuses_and_surface, on='SpecID')

    # Set SpecID as the index
    wavelength_df = wavelength_df.set_index('SpecID')

    return wavelength_df

In [ ]:
def evaluate_extra_trees(df):

    # Set the Surfaces as groups
    groups = df['SurID']
    X = df.drop(['Status', 'SurID'], axis=1)
    y = df['Status']

    # Creating the Extra Trees classifier
    et = ExtraTreesClassifier(random_state=1234)
    
    # Using GroupKFold for classification tasks
    cv = GroupKFold(n_splits=10)

    scores = []
    for train_index, test_index in cv.split(X, y, groups):

        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]
        
        # Train the Extra Trees classifier
        et.fit(X_train, y_train)
        predictions = et.predict(X_test)
        
        # Evaluate the model
        score = accuracy_score(y_test, predictions)
        scores.append(score)
    
    # Displaying the results
    print(f'{et.__class__.__name__} Cross-Validation Accuracy: {np.mean(scores):.4f} +/- {np.std(scores):.4f}')

In [ ]:
wavelength_df = prepare_wavelength_df(clean_df, 'Absorbance')

In [ ]:
wavelength_df

,400.22778,400.91116,401.59454,402.27789,402.96127,403.64465,404.32803,405.01138,405.69476,406.37814,...,1794.9886,1795.672,1796.3553,1797.0387,1797.722,1798.4055,1799.0889,1799.7722,SurID,Status
SpecID,,,,,,,,,,,,,,,,,,,,,
201210-1-07,39.462860,42.711925,45.566652,48.047821,50.176210,51.972598,53.457765,54.652490,55.577550,56.253727,...,-2.064596,-1.131661,-0.068325,1.129734,2.466836,3.947304,5.575458,7.355619,201210-1,Normal
201210-1-09,33.555879,32.770986,32.224594,31.908447,31.814287,31.933857,32.258899,32.781157,33.492374,34.384292,...,4.141024,3.922734,3.609542,3.193530,2.666778,2.021368,1.249381,0.342898,201210-1,Normal
201210-1-12,5.422334,5.210023,5.086254,5.048478,5.094145,5.220705,5.425610,5.706308,6.060250,6.484886,...,3.457851,2.799674,1.980633,0.990578,-0.180641,-1.543174,-3.107172,-4.882784,201210-1,Normal
201210-1-13,22.825529,19.572872,16.656397,14.064457,11.785405,9.807595,8.119381,6.709116,5.565154,4.675848,...,-3.864785,-4.310035,-4.783251,-5.286623,-5.822345,-6.392606,-6.999599,-7.645517,201210-1,Normal
201210-1-14,-4.960245,-4.176584,-3.370628,-2.541222,-1.687209,-0.807434,0.099260,1.034029,1.998028,2.992415,...,-4.175484,-4.089443,-3.978076,-3.841589,-3.680186,-3.494071,-3.283450,-3.048526,201210-1,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
210526-3-43,0.379089,0.820555,1.306570,1.835455,2.405532,3.015121,3.662543,4.346120,5.064172,5.815020,...,3.449541,3.305316,3.084638,2.782083,2.392228,1.909648,1.328920,0.644618,210526-3,Hyperglycemia
210526-3-44,6.072414,6.525358,6.995686,7.482422,7.984588,8.501207,9.031301,9.573893,10.128007,10.692664,...,4.504030,5.441679,6.445689,7.516708,8.655387,9.862372,11.138315,12.483863,210526-3,Hyperglycemia
210526-3-45,11.003178,12.008451,12.879986,13.624553,14.248922,14.759862,15.164145,15.468539,15.679816,15.804744,...,6.671003,7.197072,7.711103,8.211164,8.695324,9.161650,9.608210,10.033072,210526-3,Hyperglycemia


In [ ]:
evaluate_extra_trees(wavelength_df)

ExtraTreesClassifier Cross-Validation Accuracy: 0.5821 +/- 0.1750
